## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,53.22,62,75,18.41,broken clouds
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,-16.60,69,0,9.22,clear sky
2,2,Saskylakh,RU,71.9167,114.0833,-22.50,100,94,2.86,overcast clouds
3,3,Hermanus,ZA,-34.4187,19.2345,60.80,85,100,14.27,light rain
4,4,Carnarvon,AU,-24.8667,113.6333,80.67,61,100,5.75,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("Please enter the maximum temperature that you would like on your vacation: "))
min_temp = float(input("Please enter the minimum temperature that you would like on your vacation: "))


Please enter the maximum temperature that you would like on your vacation: 85
Please enter the minimum temperature that you would like on your vacation: 65


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vaca_pref_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
vaca_pref_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Carnarvon,AU,-24.8667,113.6333,80.67,61,100,5.75,overcast clouds
7,7,Ponta Do Sol,PT,32.6667,-17.1000,65.01,70,43,6.55,scattered clouds
8,8,Itaituba,BR,-4.2761,-55.9836,84.36,84,40,8.05,scattered clouds
9,9,Cayenne,GF,4.9333,-52.3333,82.44,1,75,14.97,broken clouds
12,12,Port Alfred,ZA,-33.5906,26.8910,69.40,83,28,16.87,scattered clouds
15,15,Butaritari,KI,3.0707,172.7902,80.01,74,87,14.45,overcast clouds
21,21,Sur,OM,22.5667,59.5289,82.18,37,0,6.69,clear sky
23,23,Alyangula,AU,-13.8483,136.4192,73.27,100,90,0.00,overcast clouds
25,25,Jatai,BR,-17.8814,-51.7144,80.74,65,82,4.63,broken clouds
26,26,Arraial Do Cabo,BR,-22.9661,-42.0278,84.16,61,0,23.02,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
vaca_pref_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
vaca_pref_clean_df = vaca_pref_df.dropna(how='any')
vaca_pref_clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vaca_pref_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Carnarvon,AU,80.67,overcast clouds,-24.8667,113.6333,
7,Ponta Do Sol,PT,65.01,scattered clouds,32.6667,-17.1000,
8,Itaituba,BR,84.36,scattered clouds,-4.2761,-55.9836,
9,Cayenne,GF,82.44,broken clouds,4.9333,-52.3333,
12,Port Alfred,ZA,69.40,scattered clouds,-33.5906,26.8910,
15,Butaritari,KI,80.01,overcast clouds,3.0707,172.7902,
21,Sur,OM,82.18,clear sky,22.5667,59.5289,
23,Alyangula,AU,73.27,overcast clouds,-13.8483,136.4192,
25,Jatai,BR,80.74,broken clouds,-17.8814,-51.7144,
26,Arraial Do Cabo,BR,84.16,clear sky,-22.9661,-42.0278,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
found = 0
errored = 0

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
        found += 1
    except (IndexError):
        print(f"{row['City']}: Hotel not found...skipping.")
        errored += 1
print('--------------------')
print(f'Search Complete \
    Hotels Found: {found}\
    No Hotel found: {errored}')
print('--------------------')

Tessalit: Hotel not found...skipping.
Adrar: Hotel not found...skipping.
Micheweni: Hotel not found...skipping.
Maridi: Hotel not found...skipping.
Dekar: Hotel not found...skipping.
Tadine: Hotel not found...skipping.
Angoram: Hotel not found...skipping.
Simbahan: Hotel not found...skipping.
Poum: Hotel not found...skipping.
Kalakamati: Hotel not found...skipping.
Beloha: Hotel not found...skipping.
Umm Kaddadah: Hotel not found...skipping.
--------------------
Search Complete     Hotels Found: 278    No Hotel found: 12
--------------------


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.count()

City                   290
Country                290
Max Temp               290
Current Description    290
Lat                    290
Lng                    290
Hotel Name             290
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Country</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))